In [27]:
# Depencencies and setup
import pandas as pd
import numpy as np

In [40]:
# Read and have a peek at our raw data
philly_df = pd.read_csv('C:/data/crime_date_raw/Philadelphia/crime_data_philadelphia.csv.gz')

philly_df.head()

,Dc_Dist,Psa,Dispatch_Date_Time,Dispatch_Date,Dispatch_Time,Hour,Dc_Key,Location_Block,UCR_General,Text_General_Code,Police_Districts,Month,Lon,Lat
0,18,3,2009-10-02 14:24:00,2009-10-02,14:24:00,14,200918067518,S 38TH ST / MARKETUT ST,800.0,Other Assaults,NaN,2009-10,NaN,NaN
1,14,1,2009-05-10 00:55:00,2009-05-10,00:55:00,0,200914033994,8500 BLOCK MITCH,2600.0,All Other Offenses,NaN,2009-05,NaN,NaN
2,25,J,2009-08-07 15:40:00,2009-08-07,15:40:00,15,200925083199,6TH CAMBRIA,800.0,Other Assaults,NaN,2009-08,NaN,NaN
3,35,D,2009-07-19 01:09:00,2009-07-19,01:09:00,1,200935061008,5500 BLOCK N 5TH ST,1500.0,Weapon Violations,20.0,2009-07,-75.130477,40.036389
4,9,R,2009-06-25 00:14:00,2009-06-25,00:14:00,0,200909030511,1800 BLOCK WYLIE ST,2600.0,All Other Offenses,8.0,2009-06,-75.166350,39.969532


In [12]:
len(philly_df)

2237605

In [13]:
philly_df['Dispatch_Date'].min()

'2006-01-01'

In [14]:
philly_df['Dispatch_Date'].max()

'2017-03-23'

In [16]:
# Get the categories from this data set 
cats = philly_df['Text_General_Code'].unique()
cats_df = pd.DataFrame({'Text_General_Code' : cats})
cats_df.to_csv('categories.csv')

# Here, we stop and edit 'category_keys.csv' in MS Excel...

---

In [17]:
# Merge our new column into our data set
cat_keys = pd.read_csv('category_keys.csv')
philly_short_cats = pd.merge(philly_df, cat_keys, on='Text_General_Code')

philly_short_cats.head()

,Dc_Dist,Psa,Dispatch_Date_Time,Dispatch_Date,Dispatch_Time,Hour,Dc_Key,Location_Block,UCR_General,Text_General_Code,Police_Districts,Month,Lon,Lat,Category Final
0,18,3,2009-10-02 14:24:00,2009-10-02,14:24:00,14,200918067518,S 38TH ST / MARKETUT ST,800.0,Other Assaults,NaN,2009-10,NaN,NaN,AGGRAVATED ASSAULT
1,25,J,2009-08-07 15:40:00,2009-08-07,15:40:00,15,200925083199,6TH CAMBRIA,800.0,Other Assaults,NaN,2009-08,NaN,NaN,AGGRAVATED ASSAULT
2,23,K,2009-02-10 14:33:00,2009-02-10,14:33:00,14,200923006310,2200 BLOCK RIDGE AVE,800.0,Other Assaults,16.0,2009-02,-75.171149,39.979586,AGGRAVATED ASSAULT
3,1,J,2009-02-09 22:52:00,2009-02-09,22:52:00,22,200901004250,2700 BLOCK / 2720 / 2729 WOLF ST,800.0,Other Assaults,NaN,2009-02,NaN,NaN,AGGRAVATED ASSAULT
4,24,2,2015-08-23 01:58:00,2015-08-23,01:58:00,1,201524080438,3000 BLOCK B ST,800.0,Other Assaults,17.0,2015-08,-75.125869,39.995626,AGGRAVATED ASSAULT


In [22]:
crimes = philly_short_cats['Category Final'].unique()

# Add columns for each crime category, 0 = crime is not this category, 1 = crime is this category
for c in crimes:
    philly_short_cats[c] = philly_short_cats['Category Final'].apply(lambda s: int(s==c))
    
# Group by date and export the counts to csv
philly_by_date = philly_short_cats.groupby(['Dispatch_Date'])
philly_by_date[crimes].sum().to_csv('philly_by_date.csv')

# Group by hour of the day and export the counts to csv
philly_by_hour = philly_short_cats.groupby(['Hour'])
philly_by_hour[crimes].sum().to_csv('philly_by_hour.csv')

---

In [42]:
# Clean the data further to see seasonal effects
philly_by_season = pd.read_csv('philly_by_date.csv')

# Assume date format is yyyy-mm-dd, drop the year, also return nan if it's 2017 since we don't have a full year
def clean_days(d):
    if '2017' in d:
        return np.nan
    else:
        return d[5:]

philly_by_season['Day'] = philly_by_season['Dispatch_Date'].apply(clean_days)
philly_by_season.dropna(inplace=True)
philly_by_season = philly_by_season.groupby('Day')

# This gives us the average number of each crime category committed on each date
n_years = 2017 - 2006
philly_by_season.sum().apply(lambda div_year: div_year / n_years).to_csv('philly_by_season_2006_to_2016.csv')